# Descripción del proyecto
Has recibido una tarea analítica de una tienda en línea internacional. Tus predecesores no consiguieron completarla: lanzaron una prueba A/B y luego abandonaron (para iniciar una granja de sandías en Brasil). Solo dejaron las especificaciones técnicas y los resultados de las pruebas.

## Descripción técnica
- Nombre de la prueba: recommender_system_test
- Grupos: А (control), B (nuevo embudo de pago)
- Fecha de lanzamiento: 2020-12-07
- Fecha en la que dejaron de aceptar nuevos usuarios: 2020-12-21
- Fecha de finalización: 2021-01-01
- Audiencia: 15% de los nuevos usuarios de la región de la UE
- Propósito de la prueba: probar cambios relacionados con la introducción de un sistema de recomendaciones mejorado
- Resultado esperado: dentro de los 14 días posteriores a la inscripción, los usuarios mostrarán una mejor conversión en vistas de la página del producto (el evento product_page), instancias de agregar artículos al carrito de compras (product_cart) y compras (purchase). - En cada etapa del embudo product_page → product_cart → purchase, habrá al menos un 10% de aumento.
- Número previsto de participantes de la prueba: 6 000

## Descripción de los datos
Estructura de los CSV:
- **ab_project_marketing_events_us.csv** — el calendario de eventos de marketing para 2020
- **final_ab_new_users_upd_us.csv** — todos los usuarios que se registraron en la tienda en línea desde el 7 hasta el 21 de diciembre de 2020
- **final_ab_events_upd_us.csv** — todos los eventos de los nuevos usuarios en el período comprendido entre el 7 de diciembre de 2020 y el 1 de enero de 2021
- **final_ab_participants_upd_us.csv** — tabla con los datos de los participantes de la prueba

Estructura **ab_project__marketing_events_us.csv:**
- name — el nombre del evento de marketing
- regions — regiones donde se llevará a cabo la campaña publicitaria
- start_dt — fecha de inicio de la campaña
- finish_dt — fecha de finalización de la campaña

Estructura **final_ab_new_users_upd_us.csv:**
- user_id
- first_date — fecha de inscripción
- region
- device — dispositivo utilizado para la inscripción

Estructura **final_ab_events_upd_us.csv:**
- user_id
- event_dt — fecha y hora del evento
- event_name — nombre del tipo de evento
- details — datos adicionales sobre el evento (por ejemplo, el pedido total en USD para los eventos purchase)

Estructura **final_ab_participants_upd_us.csv:**
- user_id
- ab_test — nombre de la prueba
- group — el grupo de prueba al que pertenecía el usuario


# Objetivo del estudio
Analizar y compruebar si se ha realizado correctamente el estudio de **factibilidad de una granja de sandías en Brasil**; aplicar las pruebas necesarias y analizar los resultados.


# Análisis Explatorio

## Análisis Inicial

In [2]:
#Importado de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Carga de los archivos CSV
mkt_events = pd.read_csv('datasets/ab_project_marketing_events_us.csv')
events = pd.read_csv('datasets/final_ab_events_upd_us.csv')
new_users = pd.read_csv('datasets/final_ab_new_users_upd_us.csv')
participants = pd.read_csv('datasets/final_ab_participants_upd_us.csv')

In [ ]:
#Exploración de los datos del DF mkt_events
mkt_events.info()
mkt_events.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


,name,regions,start_dt,finish_dt
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03


In [21]:
#Exploración de los datos del df events
events.info()
events.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB


,user_id,event_dt,event_name,details
173798,E85C0D938C14D781,2020-12-17 22:27:05,product_page,NaN
319258,28F350E59A67F7EB,2020-12-17 09:50:30,login,NaN
127235,A9734BA3067C98BB,2020-12-08 21:35:43,product_page,NaN
3402,DFA19F895A2AAB36,2020-12-09 22:00:14,purchase,4.99
11600,C6DD204948B0A16D,2020-12-13 00:43:57,purchase,4.99


In [9]:
#Exploración de los datos del DF new_users
new_users.info()
new_users.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


,user_id,first_date,region,device
42985,7C22AFA05B08D9FA,2020-12-18,CIS,Android
44004,151FCFDFA64C6C94,2020-12-12,EU,iPhone
44825,FA6B5071FCE56FAC,2020-12-12,EU,PC
33967,E460AD40A8316D83,2020-12-10,EU,Android
30861,9B1EA1A205864EB2,2020-12-23,EU,Android


In [10]:
#Exploración de los datos del DF participants
participants.info()
participants.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


,user_id,group,ab_test
13518,EFB19607608B9096,A,interface_eu_test
9827,5BA76F8C4411E66C,B,interface_eu_test
9327,F5122CC5F4E13616,A,interface_eu_test
384,F03F667009F16326,A,recommender_system_test
11226,2A0199F020A0A4C5,A,interface_eu_test


    - AA

## Identificación de datos nulos y repetidos

In [19]:
#Contamos los celdas repetidas y nulas en cada columna de mkt_events
print('MKT Events')
print('Celdas repetidas:\n', mkt_events.duplicated().sum())
print('Celdas nulas:\n',mkt_events.isnull().sum())

#Contamos los celdas repetidas y nulas en cada columna de events
print('\nEvents')
print('Celdas repetidas:\n', events.duplicated().sum())
print('Celdas nulas:\n',events.isnull().sum())

#Contamos los celdas repetidas y nulas en cada columna de new_users
print('\nNew Users')
print('Celdas repetidas:\n', new_users.duplicated().sum())
print('Celdas nulas:\n',new_users.isnull().sum())

#Contamos los celdas repetidas y nulas en cada columna de participants
print('\nParticipants')
print('Celdas repetidas:\n', participants.duplicated().sum())
print('Celdas nulas:\n',participants.isnull().sum())

MKT Events
Celdas repetidas:
 0
Celdas nulas:
 name         0
regions      0
start_dt     0
finish_dt    0
dtype: int64

Events
Celdas repetidas:
 0
Celdas nulas:
 user_id            0
event_dt           0
event_name         0
details       363447
dtype: int64

New Users
Celdas repetidas:
 0
Celdas nulas:
 user_id       0
first_date    0
region        0
device        0
dtype: int64

Participants
Celdas repetidas:
 0
Celdas nulas:
 user_id    0
group      0
ab_test    0
dtype: int64


## Transformación de los datos

In [33]:
#Convertimos a float la columnna 'details' del df events
#Antes de la conversión, reemplazamos los valores NaN por 0
events['details'] = events['details'].fillna(0)
events['details'] = events['details'].astype(float)


#Convertimos a datetime (año/mes/día) la columna 'first_date' del df new_users; 'start_dt' y 'finish_dt' del df mkt_events 
new_users['first_date'] = pd.to_datetime(new_users['first_date'], format='%Y-%m-%d')
mkt_events['start_dt'] = pd.to_datetime(mkt_events['start_dt'], format='%Y-%m-%d')
mkt_events['finish_dt'] = pd.to_datetime(mkt_events['finish_dt'], format='%Y-%m-%d')

#Convertimos a datetime (año/mes/día h:m:s) la columna 'event_dt' del df events
events['event_dt'] = pd.to_datetime(events['event_dt'], format='%Y-%m-%d %H:%M:%S')

#Comprobamos que las conversiones se realizaron correctamente
print('\nComprobación de conversiones:')
print('Events - event_dt:', events['event_dt'].dtype)
print('MKT Events - start_dt:', mkt_events['start_dt'].dtype)
print('MKT Events - finish_dt:', mkt_events['finish_dt'].dtype)
print('New Users - first_date:', new_users['first_date'].dtype) 


Comprobación de conversiones:
Events - event_dt: datetime64[ns]
MKT Events - start_dt: datetime64[ns]
MKT Events - finish_dt: datetime64[ns]
New Users - first_date: datetime64[ns]


# Estudio del embudo

In [ ]:
# a